Basic Statistics:

    Total Kills (K) per player. (By scope, by side, ) 
    Total Deaths (D) per player.
    Kill/Death ratio (K/D) per player.
    Average Combat Score (ACS) per player.
    Average Damage per Round (ADR) per player.

Performance Metrics:

    Headshot Percentage (HS%) per player.
    First Kills (FK) and First Deaths (FD) counts and ratios.
    Kill/Death difference (+/-) per player.
    Kill/Death difference in pistol rounds (+/-) per player.

Agent-Specific Metrics:

    Agent popularity and performance analysis.
    Win rates and performance on specific maps for each agent.

Team Performance:

    Total team Kills, Deaths, and K/D ratio.
    Team Average Combat Score (ACS) and Average Damage per Round (ADR).
    Team First Kills (FK) and First Deaths (FD) counts and ratios.

Map Performance:

    Individual player performance on each map.
    Team performance on each map.
    Win rates on each map.

Stage and Series Analysis:

    Performance in different stages (e.g., Playoffs, Grand Final).
    Series-level statistics and win rates.

Comparative Analysis:

    Compare individual player statistics against the team average.
    Compare agent performance across players.

Trend Analysis:

    Analyze how player performance changes over different maps or series.
    Identify trends in individual player K/D ratios.

Impact Analysis:

    Assess the impact of individual player performance on team success.


In [1]:
import pandas as pd
from utils import *

In [5]:
data = pd.read_csv('Data/stat_match_ascension.csv')
report = number_action_by_player_by_scope(df = data, action='K', average=True, team_name="Gentle Mates", side='Total')

In [7]:
data.head(5)

,Unnamed: 0,Player Name,Agent Name,R,ACS,K,D,K/D +/–,+/–,ADR,HS%,FK,FK/FD +/–,+/–.1,Team Name,Map Name,Map #,Stage,Series
0,0,MAGNUM,Killjoy,0.98\n0.98\n0.98,225\n237\n206,15\n10\n5,\n\n14\n9\n5\n\n,1\n1\n0,+1\n+1\n0,153\n164\n134,24%\n28%\n19%,5\n4\n1,3\n1\n2,+2\n+3\n-1,Apeks,Haven,1,Playoffs,Grand Final
1,1,Enzo,Sova,0.94\n0.75\n1.26,191\n132\n293,10\n4\n6,\n\n14\n8\n6\n\n,10\n5\n5,-4\n-4\n0,123\n100\n163,18%\n21%\n15%,2\n0\n2,2\n2\n0,0\n-2\n+2,Apeks,Haven,1,Playoffs,Grand Final
2,2,ShadoW,Breach,0.85\n0.93\n0.73,165\n174\n152,12\n9\n3,\n\n14\n8\n6\n\n,11\n9\n2,-2\n+1\n-3,89\n92\n84,23%\n35%\n8%,1\n1\n0,1\n1\n0,0\n0\n0,Apeks,Haven,1,Playoffs,Grand Final
3,3,Keiko,Jett,0.84\n1.13\n0.35,214\n278\n107,14\n12\n2,\n\n16\n9\n7\n\n,3\n2\n1,-2\n+3\n-5,151\n191\n81,26%\n27%\n22%,2\n1\n1,0\n0\n0,+2\n+1\n+1,Apeks,Haven,1,Playoffs,Grand Final
4,4,Mistic,Omen,0.70\n0.53\n0.98,130\n117\n155,8\n4\n4,\n\n15\n10\n5\n\n,4\n1\n3,-7\n-6\n-1,93\n79\n117,24%\n19%\n33%,1\n1\n0,2\n1\n1,-1\n0\n-1,Apeks,Haven,1,Playoffs,Grand Final


In [18]:
data_player = data[data['Player Name'] == "TakaS"]

set_agent_played = list(set(data_player["Agent Name"]))

In [20]:
agent_popularity = {
        'player': "player_name",
        'played_agent' : {agent:0 for agent in list(set(data_player["Agent Name"]))}
    }

In [27]:
r = agent_popularity(data, "TakaS")

In [26]:
# needs to change the other by agent 
# Agent-Specific Metrics:

#     Agent popularity and performance analysis.
#     Win rates and performance on specific maps for each agent.

def win_rates_on_map_by_agent(data, map_name, agent):
    #Win rates and performance on specific maps for each agent.

    try:
        data_map = data[data["Map Name"] == map_name]
        data_map_by_agent = data_map[data_map["Agent Name"] == agent]
        # the win needs to be added in the scrapper
    except NameError:
        print(f'{NameError}')
